# Modelos de Regressão

Este notebook investiga diferentes abordagens de regressão para análise de dados, com foco nos modelos de Regressão Linear e Lasso. São aplicadas técnicas de pré-processamento, como a transformação de Box-Cox, visando aprimorar a modelagem. A avaliação dos modelos é feita com base em métricas como RMSE e R², considerando tanto o conjunto completo de dados quanto versões filtradas após a remoção de outliers das questões do Enem (vetorizadas).

## Apoio


In [89]:
# Importando Dependências para Modelos de Regressão
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import train_test_split

In [ ]:
# Leitura do dataset
df_enem = pd.read_pickle('../data/final/enem_data_embeddings.pkl')
df_enem.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,alternativa_correta,enunciado_embbedings_word2vec,gabarito_embbedings_word2vec
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,"[-0.030405086, 0.022974016, -0.027739665, -0.0...","[-0.12797333, -0.114192665, 0.13532682, -0.057..."
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,"[-0.03320758, 0.03166411, -0.04888896, 0.01524...","[-0.18022183, 0.041319773, -0.06070747, -0.069..."
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,"[-0.0063855667, 0.022534302, -0.014108029, -0....","[-0.049211252, 0.073626995, 0.013877999, 0.085..."
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,"[-0.014004647, 0.02424562, -0.010280999, 0.010...","[-0.06145667, 0.129228, -0.004840499, 0.033785..."
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,"[-0.013194435, 0.01645982, -0.0011703374, -0.0...","[0.045796998, -0.120854005, 0.06641175, -0.123..."


In [79]:
# Removendo o ano de 2021
df_enem_aux = df_enem.drop(df_enem[df_enem['ANO'] == 2021].index).reset_index()
df_enem_aux = df_enem_aux[df_enem_aux['nu_param_B'].notna()]

## Regressão Linear

### Dataset Completo

In [113]:
# Separando os dados
X = [np.array(embedding) for embedding in df_enem_aux['enunciado_embbedings_word2vec']]
y = df_enem_aux['nu_param_B']

In [114]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [115]:
# Separando em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [116]:
# Adicionando constante
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [117]:
model = sm.OLS(y_train, X_train).fit()
print(model.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 12 May 2025   Prob (F-statistic):                nan
Time:                        18:55:42   Log-Likelihood:                 5924.3
No. Observations:                 186   AIC:                        -1.148e+04
Df Residuals:                       0   BIC:                        -1.088e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.1110        inf          0        n

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


In [112]:
# Calculando o RMSE
pred = model.predict(X_test)

rms = root_mean_squared_error(y_test, pred)

print("RMSE com todas as variáveis:", rms)

RMSE com todas as variáveis: 0.804569667829403


## Regressão Lasso

### Dataset Completo

In [124]:
# Separando os dados
X = [np.array(embedding) for embedding in df_enem_aux['enunciado_embbedings_word2vec']]
y = df_enem_aux['nu_param_B']

In [125]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [126]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [127]:
model_lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=42).fit(
    X_train, y_train
)
pred_lasso = model_lasso.predict(X_test)

print("melhor r2 score (treino):", model_lasso.score(X_train, y_train))
print("r2 score (teste):", model_lasso.score(X_test, y_test))
print("melhor alpha:", model_lasso.alpha_)
print("RMSE com o alpha escolhido:", root_mean_squared_error(y_test, pred_lasso))

melhor r2 score (treino): 0.40448353255284286
r2 score (teste): -0.15117102602594779
melhor alpha: 0.001
RMSE com o alpha escolhido: 0.5082764208689048


### Modelo com corte na dificuldade (-3, 3)

In [128]:
enem_filtered = df_enem_aux.copy()
enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

,nu_param_B
count,263.000000
mean,1.075595
std,0.661348
min,-0.726450
25%,0.612925
50%,1.051180
75%,1.535350
max,2.802770


In [129]:
# Separando os dados
X = [np.array(embedding) for embedding in enem_filtered['enunciado_embbedings_word2vec']]
y = enem_filtered['nu_param_B']

In [130]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [131]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [134]:
model_lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=42).fit(
    X_train, y_train
)
pred_lasso = model_lasso.predict(X_test)

print("melhor r2 score (treino):", model_lasso.score(X_train, y_train))
print("r2 score (teste):", model_lasso.score(X_test, y_test))
print("melhor alpha:", model_lasso.alpha_)
print("RMSE com o alpha escolhido:", root_mean_squared_error(y_test, pred_lasso))

melhor r2 score (treino): 0.43424444830997866
r2 score (teste): 0.011537872316580211
melhor alpha: 0.001
RMSE com o alpha escolhido: 0.599207064043015
